# FlowSOM Analysis Pipeline - Notebook Headless

## Pipeline complète d'analyse FlowSOM pour données de cytométrie en flux

Ce notebook "miroir" de l'application FlowSOM Analyzer permet:
- **Debug & Introspection**: Visualiser les données à chaque étape
- **Tuning rapide**: Tester différents paramètres sans relancer l'app
- **Séparation des responsabilités**: Logique métier pure, sans UI

---

**Auteur**: Florian Magne
**Version**: 1.0
**Date**: Janvier 2026

## 1. Import des Librairies

Import de toutes les librairies nécessaires avec vérification de disponibilité des dépendances optionnelles.

In [ ]:
# -*- coding: utf-8 -*-

# IMPORTS début du fichier
import sys
import warnings
from pathlib import Path
from typing import Optional, List, Dict, Any, Tuple
from datetime import datetime

warnings.filterwarnings('ignore')

# Imports scientifiques de base
import numpy as np
import pandas as pd

# Imports visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration style matplotlib pour fond sombre (comme l'UI)
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#1e1e2e'
plt.rcParams['axes.facecolor'] = '#1e1e2e'
plt.rcParams['text.color'] = '#cdd6f4'
plt.rcParams['axes.labelcolor'] = '#cdd6f4'
plt.rcParams['xtick.color'] = '#cdd6f4'
plt.rcParams['ytick.color'] = '#cdd6f4'
plt.rcParams['figure.figsize'] = (12, 8)

# Plotly pour visualisations interactives
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    # Configuration pour affichage dans les notebooks
    pio.renderers.default = 'notebook'
    PLOTLY_AVAILABLE = True
    print("✅ Plotly disponible")
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly non installé (optionnel): pip install plotly")

# IMPORTS flowsom et anndata, l'un est le package d'analyse du FlowSOM, l'autre est pour gérer les données dans des objets AnnData
try:
    import flowsom as fs
    import anndata as ad
    FLOWSOM_AVAILABLE = True
    print("✅ FlowSOM disponible")
except ImportError:
    FLOWSOM_AVAILABLE = False
    print("❌ FlowSOM non installé: pip install flowsom")

# Import de Scanpy pour UMAP/t-SNE
try:
    import scanpy as sc
    SCANPY_AVAILABLE = True
    print("✅ Scanpy disponible")
except ImportError:
    SCANPY_AVAILABLE = False
    print("⚠️ Scanpy non installé (optionnel): pip install scanpy")

# Import de UMAP
try:
    import umap
    UMAP_AVAILABLE = True
    print("✅ UMAP disponible")
except ImportError:
    UMAP_AVAILABLE = False
    print("⚠️ UMAP non installé (optionnel): pip install umap-learn")


# Import de t-SNE via sklearn car t-SNE trop lent à être implémenté dans Scanpy (et FlowSOM)
try:
    from sklearn.manifold import TSNE
    from sklearn.metrics import silhouette_score
    from sklearn.cluster import AgglomerativeClustering
    SKLEARN_AVAILABLE = True
    print("✅ Scikit-learn disponible")
except ImportError:
    SKLEARN_AVAILABLE = False
    print("⚠️ Scikit-learn non installé: pip install scikit-learn")

# FlowKit pour transformations Logicle
try:
    import flowkit as fk
    FLOWKIT_AVAILABLE = True
    print("✅ FlowKit disponible (transformations Logicle précise en 1 fonction)")
except ImportError:
    FLOWKIT_AVAILABLE = False
    print("⚠️ FlowKit non installé (optionnel): pip install flowkit")

# FCSWrite pour export FCS
try:
    import fcswrite
    FCSWRITE_AVAILABLE = True
    print("✅ FCSWrite disponible (export FCS)")
except ImportError:
    FCSWRITE_AVAILABLE = False
    print("⚠️ FCSWrite non installé (optionnel): pip install fcswrite")

# Scipy pour statistiques 
from scipy import stats

In [ ]:
# Import en haut de fichier des classes utilitaires permettant les transformations des fichiers ainsi que le pre-gating 

class DataTransformer:
    """
    Transformations de données de cytométrie (Logicle, Arcsinh, etc.).
    Classe statique réutilisable sans dépendance à l'UI.
    """
    
    @staticmethod
    def arcsinh_transform(data: np.ndarray, cofactor: float = 5.0) -> np.ndarray:
        """
        Transformation Arcsinh (inverse hyperbolic sine).
        
        Args en entrée:
            data: Matrice de données (n_cells, n_markers)
            cofactor: Facteur de division (5 pour flow cytometry)
        
        Returns:
            Données transformées
        """
        return np.arcsinh(data / cofactor)
    
    @staticmethod
    def arcsinh_inverse(data: np.ndarray, cofactor: float = 5.0) -> np.ndarray:
        """Inverse de la transformation Arcsinh."""
        return np.sinh(data) * cofactor
    
    @staticmethod
    def logicle_transform(data: np.ndarray, T: float = 262144.0, M: float = 4.5,
                          W: float = 0.5, A: float = 0.0) -> np.ndarray:
        """
        Transformation Logicle (biexponentielle).
        
        Args en entrée:
            data: Matrice de données
            T: Maximum de l'échelle linéaire (262144 = 2^18)
            M: Décades de largeur
            W: Linéarisation près de zéro
            A: Décades additionnelles (négatifs)
        
        Returns:
            Données transformées
        """
        if FLOWKIT_AVAILABLE:
            # Utiliser FlowKit si disponible (plus précis) avec une fonction prédéfinie
            try:
                xform = fk.transforms.LogicleTransform(T=T, M=M, W=W, A=A)
                return xform.apply(data)
            except:
                pass
        
        # Approximation si FlowKit absent: Arcsinh modifié
        w_val = W * np.log10(np.e)
        return np.arcsinh(data / (T / (10 ** M))) * (M / np.log(10))
    
    @staticmethod
    def log_transform(data: np.ndarray, base: float = 10.0,
                      min_val: float = 1.0) -> np.ndarray:
        """Transformation logarithmique standard."""
        data_clipped = np.maximum(data, min_val)
        return np.log(data_clipped) / np.log(base)
    
    @staticmethod
    def zscore_normalize(data: np.ndarray) -> np.ndarray:
        """Normalisation Z-score (moyenne=0, std=1)."""
        mean = np.nanmean(data, axis=0)
        std = np.nanstd(data, axis=0)
        std[std == 0] = 1  # Éviter division par zéro
        return (data - mean) / std
    
    @staticmethod
    def min_max_normalize(data: np.ndarray) -> np.ndarray:
        """Normalisation Min-Max [0, 1]."""
        min_val = np.nanmin(data, axis=0)
        max_val = np.nanmax(data, axis=0)
        range_val = max_val - min_val
        range_val[range_val == 0] = 1
        return (data - min_val) / range_val


class PreGating:
    """
    Pre-gating automatique pour la sélection des populations d'intérêt.
    Basé sur FSC/SSC pour exclure les débris et les doublets.
    """
    
    @staticmethod
    def find_marker_index(var_names: List[str], patterns: List[str]) -> Optional[int]:
        """Trouve l'index d'un marqueur parmi les patterns donnés."""
        var_upper = [v.upper() for v in var_names]
        for pattern in patterns:
            for i, name in enumerate(var_upper):
                if pattern.upper() in name:
                    return i
        return None
    
    @staticmethod
    def gate_viable_cells(X: np.ndarray, var_names: List[str],
                          min_percentile: float = 2.0, 
                          max_percentile: float = 98.0) -> np.ndarray:
        """
        Gate les cellules viables basé sur FSC/SSC.
        
        Args:
            X: Matrice des données (n_cells, n_markers)
            var_names: Liste des noms de marqueurs
            min_percentile: Percentile minimum (exclusion débris)
            max_percentile: Percentile maximum (exclusion doublets)
        
        Returns:
            Masque booléen des cellules viables
        """
        n_cells = X.shape[0]
        mask = np.ones(n_cells, dtype=bool)
        
        # Trouver FSC (priorité à FSC-A)
        fsc_idx = PreGating.find_marker_index(var_names, ['FSC-A', 'FSC-H', 'FSC'])
        if fsc_idx is not None:
            fsc_vals = X[:, fsc_idx].astype(np.float64)
            fsc_vals = np.where(np.isfinite(fsc_vals), fsc_vals, np.nan)
            low = np.nanpercentile(fsc_vals, min_percentile)
            high = np.nanpercentile(fsc_vals, max_percentile)
            mask &= np.isfinite(fsc_vals) & (fsc_vals >= low) & (fsc_vals <= high)
        
        # Trouver SSC (priorité à SSC-A)
        ssc_idx = PreGating.find_marker_index(var_names, ['SSC-A', 'SSC-H', 'SSC'])
        if ssc_idx is not None:
            ssc_vals = X[:, ssc_idx].astype(np.float64)
            ssc_vals = np.where(np.isfinite(ssc_vals), ssc_vals, np.nan)
            low = np.nanpercentile(ssc_vals, min_percentile)
            high = np.nanpercentile(ssc_vals, max_percentile)
            mask &= np.isfinite(ssc_vals) & (ssc_vals >= low) & (ssc_vals <= high)
        
        return mask
    
    @staticmethod
    def gate_singlets(X: np.ndarray, var_names: List[str],
                      ratio_min: float = 0.6, ratio_max: float = 1.5) -> np.ndarray:
        """
        Gate les singlets basé sur le ratio FSC-A/FSC-H.
        Les doublets ont typiquement un ratio > 1.3-1.5.
        
        Args:
            X: Matrice des données
            var_names: Liste des noms de marqueurs
            ratio_min: Ratio minimum acceptable
            ratio_max: Ratio maximum acceptable
        
        Returns:
            Masque booléen des singlets
        """
        n_cells = X.shape[0]
        
        fsc_a_idx = PreGating.find_marker_index(var_names, ['FSC-A'])
        fsc_h_idx = PreGating.find_marker_index(var_names, ['FSC-H'])
        
        if fsc_a_idx is None or fsc_h_idx is None:
            print("⚠️ FSC-A ou FSC-H non trouvé, pas de gating singlets")
            return np.ones(n_cells, dtype=bool)
        
        fsc_a = X[:, fsc_a_idx].astype(np.float64)
        fsc_h = X[:, fsc_h_idx].astype(np.float64)
        
        # Valeurs minimum pour éviter division par zéro
        min_val = 100
        valid_h = fsc_h > min_val
        
        ratio = np.full(n_cells, np.nan)
        ratio[valid_h] = fsc_a[valid_h] / fsc_h[valid_h]
        
        mask = np.isfinite(ratio) & (ratio >= ratio_min) & (ratio <= ratio_max)
        
        return mask
    
    @staticmethod
    def gate_cd45_positive(X: np.ndarray, var_names: List[str],
                           threshold_percentile: float = 10) -> np.ndarray:
        """
        Gate les cellules CD45+ (leucocytes).
        
        Returns:
            Masque booléen des cellules CD45+
        """
        n_cells = X.shape[0]
        
        cd45_idx = PreGating.find_marker_index(var_names, ['CD45', 'CD45-PECY5', 'CD45-PC5'])
        if cd45_idx is None:
            print("⚠️ CD45 non trouvé, pas de gating CD45+")
            return np.ones(n_cells, dtype=bool)
        
        cd45_vals = X[:, cd45_idx].astype(np.float64)
        cd45_vals = np.where(np.isfinite(cd45_vals), cd45_vals, np.nan)
        
        threshold = np.nanpercentile(cd45_vals, threshold_percentile)
        
        return np.where(np.isnan(cd45_vals), False, cd45_vals > threshold)


print("✅ Classes DataTransformer et PreGating chargées!")

## 2. Chargement des Fichiers FCS

Chargement des fichiers FCS depuis les dossiers spécifiés. 
- **Sain (NBM)**: Moelle osseuse normale (référence)
- **Pathologique**: Échantillons patients à analyser

In [ ]:
# CONFIGURATION DES CHEMINS
# Bien modifier ces chemins selon votre environnement actuel pour le bon chargement des données

# Dossier des fichiers sains (référence NBM)
HEALTHY_FOLDER = Path(r"Data/Sain")

# Dossier des fichiers pathologiques (patients)
PATHOLOGICAL_FOLDER = Path(r"Data/Patho")

# Mode d'analyse: 
# - True: Comparer Sain vs Pathologique
# - False: Analyser uniquement les fichiers pathologiques
COMPARE_MODE = False

print(f"Dossier Sain: {HEALTHY_FOLDER}")
print(f"Dossier Pathologique: {PATHOLOGICAL_FOLDER}")
print(f"Mode comparaison: {'Activé' if COMPARE_MODE else 'Patient seul'}")

In [ ]:
# FONCTIONS DE CHARGEMENT FCS

def get_fcs_files(folder: Path) -> List[str]:
    """Récupère la liste des fichiers FCS dans un dossier. Et renvoie une chaine de caractère"""
    if not folder.exists():
        print(f"⚠️ Dossier non trouvé: {folder}")
        return []
    
    files = set()
    for f in folder.glob("*.fcs"):
        files.add(str(f))
    for f in folder.glob("*.FCS"):
        files.add(str(f))
    
    return sorted(list(files))


def load_fcs_files(files: List[str], condition: str = "Unknown") -> List[ad.AnnData]:
    """
    Charge plusieurs fichiers FCS et retourne une liste d'AnnData.
    
    Args:
        files: Liste des chemins de fichiers FCS
        condition: Label de condition ("Sain" ou "Pathologique")
    
    Returns:
        Liste d'objets AnnData
    """
    # La ligne suivante crée la liste vide pour stocker les AnnData puis boucle sur chaque fichier (éviter le plantage complet)
    adatas = []
    
    for fpath in files:
        try:
            print(f"    Chargement: {Path(fpath).name}...", end=" ")
            
            # Lecture avec la fonction de base de flowsom
            adata = fs.io.read_FCS(fpath)
            
            # Ajouter les métadonnées avec un nombre de cellules qui sera égale a la forme de l'objet adata 
            n_cells = adata.shape[0]
            adata.obs['condition'] = condition # Rajoute la condition du fichier : "Sain" ou "Pathologique"
            adata.obs['file_origin'] = Path(fpath).name # Rajoute une observation avec Nom du fichier source (obs = One-dimensional annotation of observations)
            
            adatas.append(adata) # Ajoute à la liste des AnnData
            print(f"{n_cells:,} cellules")
            
        except Exception as e:
            print(f"Erreur: {e}")
    
    return adatas

# Logs sur le cahrgement des fichiers
print("="*60)
print("CHARGEMENT DES FICHIERS FCS")
print("="*60)

# Fichiers sains en fonction du mode défini
healthy_files = get_fcs_files(HEALTHY_FOLDER) if COMPARE_MODE else []
print(f"\nFichiers Sains (NBM): {len(healthy_files)}")

healthy_adatas = []
if healthy_files:
    healthy_adatas = load_fcs_files(healthy_files, condition="Sain")

# Fichiers sains en fonction du mode défini
patho_files = get_fcs_files(PATHOLOGICAL_FOLDER)
print(f"\nFichiers Pathologiques: {len(patho_files)}")

patho_adatas = []
if patho_files:
    patho_adatas = load_fcs_files(patho_files, condition="Pathologique")

# Résumé
print("\n" + "="*60)
print(f"RÉSUMÉ DU CHARGEMENT")
print(f"   Fichiers Sains chargés: {len(healthy_adatas)}")
print(f"   Fichiers Pathologiques chargés: {len(patho_adatas)}")
# Résumé a.shape = pour chaque AnnData, prend le nombre de cellules (lignes) et concatène si nécessaire
total_cells = sum([a.shape[0] for a in healthy_adatas + patho_adatas])
print(f"   Total cellules: {total_cells:,}")
print("="*60)

## 3. Exploration de la Structure des Données Brutes

Avant toute transformation, examinons la structure des données:
- Dimensions (cellules x marqueurs)
- Noms des colonnes (marqueurs)
- Types de données et plages de valeurs

In [ ]:
# CONCATÉNATION DES DONNÉES

# Combiner tous les AnnData défini dans la cellule précédente
all_adatas = healthy_adatas + patho_adatas

# Vérification
if len(all_adatas) == 0:
    raise ValueError("❌ Aucun fichier FCS chargé! Vérifiez les chemins.")

# Concaténer avec intersection des colonnes (communes à tous les fichiers) ligne par ligne
if len(all_adatas) > 1:
    combined_data = ad.concat(all_adatas, join='inner') # join='inner' pour ne garder que les marqueurs communs à changer par outer si on veut garder tous les marqueurs
else:
    combined_data = all_adatas[0].copy() # Si un seul fichier, juste copier pour éviter de mofifier l'original

print(f"Données combinées: {combined_data.shape}")
print(f"   → {combined_data.shape[0]:,} cellules")
print(f"   → {combined_data.shape[1]} marqueurs")

In [ ]:
# EXPLORATION DE LA STRUCTURE
print("="*70)
print("STRUCTURE DES DONNÉES")
print("="*70)

# Liste des marqueurs enregistré dans la varaible var_names = canaux (ici c'est bien un nom de variable)
var_names = list(combined_data.var_names)
print(f"\nMarqueurs ({len(var_names)}):")
for i, name in enumerate(var_names):
    print(f"   [{i:2d}] {name}")

# Identification des types de marqueurs car les recos indiquent d'enelever le scatter pour les analyses de clustering
print("\nClassification des marqueurs:")

#Ici le code n for n in var pose la question : "Est-ce qu'au moins UN des motifs de la liste scatter_patterns se trouve dans le nom actuel n ?"
scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']
scatter_markers = [n for n in var_names if any(p in n.upper() for p in scatter_patterns)]
fluor_markers = [n for n in var_names if n not in scatter_markers]

print(f"   Scatter/Time: {scatter_markers}")
print(f"   Fluorescence: {fluor_markers}")

# Statistiques de base
print("\nObservations (métadonnées):")
print(combined_data.obs.head(10))

In [ ]:
# CONVERSION EN DATAFRAME POUR EXPLORATION
HEADER = True
# Extraire la matrice de données
X = combined_data.X # Matrice des données (n_cells, n_markers)
if hasattr(X, 'toarray'): # Si sparse matrix, convertir en dense pour pandas
    X = X.toarray()

# Créer un DataFrame pandas pour faciliter l'exploration avec df comme commande pandas classique
df_raw = pd.DataFrame(X, columns=var_names) # Crée le DataFrame avec les noms de colonnes 
df_raw['condition'] = combined_data.obs['condition'].values # Ajoute une colonne condition
df_raw['file_origin'] = combined_data.obs['file_origin'].values # Ajoute une colonne file_origin

print("DataFrame créé pour exploration")
print(f"   Shape: {df_raw.shape}")
print("\nAperçu des données brutes:")
df_raw.head(50)

In [ ]:
# Stats descriptives marqueurs de fluorescence et scatter
print("Statistiques descriptives fluorescence")
display(df_raw[fluor_markers].describe())

print("\nStatistiques descriptives scatter")
display(df_raw[scatter_markers].describe())

## 4. Contrôle Qualité des données- Analyse des Distributions

Visualisation des distributions brutes pour identifier:
- Outliers et valeurs aberrantes
- Valeurs négatives (problème de compensation)
- NaN/Inf dans les données

In [ ]:
# Vérif des varaibles problématiques suite de l'exploration du dataset

print("ANALYSE DES DONNÉES BRUTES")
print("="*60)

# ========== MARQUEURS DE FLUORESCENCE ==========
print("\nMARQUEURS DE FLUORESCENCE")
print("-"*60)

# Vérifier NaN
nan_count = df_raw[fluor_markers].isna().sum()
print(f"\nValeurs NaN par marqueur:")
for marker, count in nan_count.items():
    if count > 0:
        print(f"   {marker}: {count:,} ({count/len(df_raw)*100:.2f}%)")
    
if nan_count.sum() == 0:
    print("   ✅ Aucun NaN détecté!")

# Vérifier Inf (valeur infinie) ex sur un post log 
inf_count = np.isinf(df_raw[fluor_markers]).sum()
print(f"\nValeurs Inf par marqueur:")
if inf_count.sum() == 0:
    print("   ✅ Aucun Inf détecté!")
else:
    for marker, count in inf_count.items():
        if count > 0:
            print(f"   {marker}: {count:,}")

# Vérifier valeurs négatives
neg_count = (df_raw[fluor_markers] < 0).sum()
print(f"\n➖ Valeurs négatives par marqueur:")
has_negatives = False
for marker, count in neg_count.items():
    if count > 0:
        has_negatives = True
        # Compter le nombre total de cellules valides (non-NaN) pour ce marqueur
        total_valid = df_raw[marker].notna().sum()
        print(f"   {marker}: {count:,} / {total_valid:,} ({count/total_valid*100:.2f}%)")
        
if not has_negatives:
    print("   ✅ Aucune valeur négative!")
else:
    print("\n   ⚠️ Les valeurs négatives peuvent indiquer un problème de compensation")
    print("   → La transformation Arcsinh ou Logicle peut les gérer")

# ========== MARQUEURS SCATTER/TIME ==========
print("\n\nMARQUEURS SCATTER/TIME")
print("-"*60)

# Vérifier NaN
nan_count_scatter = df_raw[scatter_markers].isna().sum()
print(f"\nValeurs NaN par marqueur:")
for marker, count in nan_count_scatter.items():
    if count > 0:
        print(f"   {marker}: {count:,} ({count/len(df_raw)*100:.2f}%)")
    
if nan_count_scatter.sum() == 0:
    print("   ✅ Aucun NaN détecté!")

# Vérifier Inf
inf_count_scatter = np.isinf(df_raw[scatter_markers]).sum()
print(f"\nValeurs Inf par marqueur:")
if inf_count_scatter.sum() == 0:
    print("   ✅ Aucun Inf détecté!")
else:
    for marker, count in inf_count_scatter.items():
        if count > 0:
            print(f"   {marker}: {count:,}")

# Vérifier valeurs négatives
neg_count_scatter = (df_raw[scatter_markers] < 0).sum()
print(f"\n➖ Valeurs négatives par marqueur:")
has_negatives_scatter = False
for marker, count in neg_count_scatter.items():
    if count > 0:
        has_negatives_scatter = True
        total_valid = df_raw[marker].notna().sum()
        print(f"   {marker}: {count:,} / {total_valid:,} ({count/total_valid*100:.2f}%)")
        
if not has_negatives_scatter:
    print("   ✅ Aucune valeur négative!")
else:
    print("\n   ℹ️ Les valeurs négatives dans scatter sont rares mais possibles")

print("\n" + "="*60)

In [ ]:
# Histogrammes des distributions brutes pour explorer visuellement les données

# Sélectionner les marqueurs à visualiser (max 12 pour lisibilité)
markers_to_plot = fluor_markers[:12] if len(fluor_markers) > 12 else fluor_markers  # Opérateur ternaire: prendre 12 premiers si > 12, sinon tous

n_markers = len(markers_to_plot)  # Nombre de marqueurs à afficher
n_cols = 4  # 4 colonnes par ligne
n_rows = (n_markers + n_cols - 1) // n_cols  # Calcul nb lignes (division entière arrondie vers le haut)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))  # Créer grille n_rows × n_cols (largeur 20, hauteur 5 par ligne)
axes = axes.flatten() if n_markers > 1 else [axes]  # Aplatir tableau 2D en liste 1D pour itération facile

for i, marker in enumerate(markers_to_plot):  # Boucle sur chaque marqueur avec index i
    ax = axes[i]  # Récupérer le sous-graphique i
    data = df_raw[marker].dropna()  # Extraire données du marqueur et supprimer NaN
    
    ax.hist(data, bins=100, color='#89b4fa', alpha=0.7, edgecolor='none')  # Histogramme 100 barres, bleu, 70% opacité
    ax.set_title(marker, fontsize=11, fontweight='bold')  # Titre = nom du marqueur
    ax.set_xlabel('Valeur brute')  # Label axe X
    ax.set_ylabel('Count')  # Label axe Y = nombre de cellules
    ax.axvline(0, color='#f38ba8', linestyle='--', alpha=0.5, label='Zéro')  # Ligne verticale rouge à x=0
    
    # Statistiques min/max dans une boîte en haut à droite
    ax.text(0.98, 0.95, f'min: {data.min():.0f}\nmax: {data.max():.0f}',  # Texte avec stats
            transform=ax.transAxes, ha='right', va='top', fontsize=8,  # Coordonnées relatives (0-1), alignement
            bbox=dict(boxstyle='round', facecolor='#313244', alpha=0.8))  # Boîte grise arrondie semi-transparente

# Cacher les axes vides (si 10 marqueurs sur grille 3×4, cacher les 2 dernières cases)
for i in range(n_markers, len(axes)):
    axes[i].set_visible(False)

plt.suptitle('Distributions Brutes des Marqueurs (avant transformation)',  # Titre général
             fontsize=14, fontweight='bold', y=1.02)  # Décalé vers le haut
plt.tight_layout()  # Ajuster espacement auto
plt.show()  # Afficher

In [ ]:
# Histogrammes des marqueurs SCATTER/TIME pour exploration visuelle

# Sélectionner tous les marqueurs scatter (généralement peu nombreux)
scatter_to_plot = scatter_markers  # FSC, SSC, TIME

n_scatter = len(scatter_to_plot)  # Nombre de marqueurs scatter
n_cols_scatter = min(3, n_scatter)  # Max 3 colonnes pour les scatter
n_rows_scatter = (n_scatter + n_cols_scatter - 1) // n_cols_scatter  # Calcul nb lignes

fig, axes = plt.subplots(n_rows_scatter, n_cols_scatter, figsize=(18, 6*n_rows_scatter))  # Grille pour scatter (largeur 18, hauteur 6 par ligne)
axes = axes.flatten() if n_scatter > 1 else [axes]  # Aplatir en liste 1D

for i, marker in enumerate(scatter_to_plot):  # Boucle sur chaque marqueur scatter
    ax = axes[i]  # Sous-graphique i
    data = df_raw[marker].dropna()  # Données sans NaN
    
    ax.hist(data, bins=100, color='#a6e3a1', alpha=0.7, edgecolor='none')  # Vert pour différencier
    ax.set_title(marker, fontsize=12, fontweight='bold')  # Titre
    ax.set_xlabel('Valeur brute')  # Axe X
    ax.set_ylabel('Count')  # Axe Y
    
    # Statistiques complètes
    ax.text(0.02, 0.95, f'min: {data.min():.0f}\nmax: {data.max():.0f}\nmean: {data.mean():.0f}\nmedian: {data.median():.0f}',
            transform=ax.transAxes, ha='left', va='top', fontsize=8,
            bbox=dict(boxstyle='round', facecolor='#313244', alpha=0.8))

# Cacher axes vides
for i in range(n_scatter, len(axes)):
    axes[i].set_visible(False)

plt.suptitle('Distributions Scatter/Time (FSC, SSC, TIME)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Visualisation Interactive avec FlowKit + Bokeh

Utilisation native de FlowKit pour visualisations interactives :
- **Histogrammes** avec bins/ranges personnalisables
- **Scatter plots** interactifs avec zoom/pan
- **Contour plots** avec densité
- Rendu Bokeh pour l'interactivité (zoom, pan, hover)

📚 Documentation : https://flowkit.readthedocs.io/en/latest/index.html
https://www.frontiersin.org/journals/immunology/articles/10.3389/fimmu.2021.768541/full

---

### ⚠️ IMPORTANT : Nomenclature FCS

**FlowKit utilise les PnN labels** (noms techniques), pas les PnS (descriptions).

- **PnN** = Nom technique (ex: `'Horizon V500-A'`) ← À utiliser
- **PnS** = Description bio (ex: `'CD45 KO'`) ← Non utilisable

**Exemple :** Pour CD45, utiliser `'Horizon V500-A'` (pas `'CD45 KO'`).

Exécutez la cellule suivante pour voir la correspondance PnN ↔ PnS.


In [ ]:
# Initialisation FlowKit et Bokeh + Création du Sample

if not FLOWKIT_AVAILABLE:
    fk_sample = None
else:
    try:
        from bokeh.plotting import show, output_notebook
        from bokeh.io import export_png
        output_notebook()
    except ImportError:
        pass
    
    # Utiliser les fichiers FCS déjà identifiés dans le notebook
    all_fcs_files = healthy_files + patho_files
    
    if all_fcs_files:
        example_fcs = str(all_fcs_files[0])
        fk_sample = fk.Sample(example_fcs)
    else:
        fk_sample = None

In [ ]:
# AFFICHER LES NOMS DE CANAUX EXACTS DU FICHIER FCS

if fk_sample is not None:
    print("="*80)
    print(f"CANAUX DU FICHIER FCS: {Path(example_fcs).name}")
    print("="*80)
    
    print(f"\nPnN Labels ({len(fk_sample.pnn_labels)} canaux) - NOMS À UTILISER DANS FLOWKIT:")
    print("-"*80)
    for i, label in enumerate(fk_sample.pnn_labels, 1):
        print(f"   [{i:2d}] '{label}'")
    
    # Afficher aussi les PnS labels (descriptions) si disponibles
    print(f"\n\nPnS Labels (descriptions):")
    print("-"*80)
    for i, label in enumerate(fk_sample.pns_labels, 1):
        print(f"   [{i:2d}] {label}")
    
else:
    print("⚠️ FlowKit Sample non chargé")

In [ ]:
# Définir les colonnes FSC et SSC pour les visualisations ultérieures
fsc_col = next((c for c in var_names if 'FSC-A' in c.upper() or 'FSC' in c.upper()), None)
ssc_col = next((c for c in var_names if 'SSC-A' in c.upper() or 'SSC' in c.upper()), None)

if fsc_col:
    print(f"✅ FSC détecté: {fsc_col}")
if ssc_col:
    print(f"✅ SSC détecté: {ssc_col}")

In [ ]:
# Histogramme basique FlowKit

if fk_sample is not None:
    CHANNEL = 'Horizon V500-A' #changer ici le channel a afficher en fonction de count
    p = fk_sample.plot_histogram(CHANNEL, source='raw', bins=256)
    show(p)

In [ ]:
# Scatter plot 2D interactif

if fk_sample is not None:
    x_channel = 'Horizon V500-A'
    y_channel = 'PE-A'
    p = fk_sample.plot_scatter(x_channel, y_channel, source='raw')
    show(p)

In [ ]:
# Histogramme 1D interactif avec Plotly (zoom, pan, hover)
# Sélectionner un marqueur à visualiser (modifiable)
MARKER_TO_PLOT = 'CD45 KO'  # Changer ici le nom exact du marqueur à visualiser

print(f"Visualisation: {MARKER_TO_PLOT}")

# Extraire les données
marker_data = df_raw[MARKER_TO_PLOT].dropna().values

# Importer plotly pour l'interactivité
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.io as pio
    
    # Configurer le renderer pour Jupyter (évite l'erreur nbformat)
    try:
        pio.renderers.default = 'notebook'
    except:
        try:
            pio.renderers.default = 'jupyterlab'
        except:
            pio.renderers.default = 'browser'
    
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly non installé - pip install plotly")

if PLOTLY_AVAILABLE:
    # Créer une figure avec 4 subplots (2x2)
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'{MARKER_TO_PLOT} - Brut (Linéaire)',
            f'{MARKER_TO_PLOT} - Arcsinh (cofactor=5)',
            f'{MARKER_TO_PLOT} - Logicle/Arcsinh',
            f'{MARKER_TO_PLOT} - Log10'
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Données brutes
    fig.add_trace(
        go.Histogram(x=marker_data, nbinsx=200, name='Brut',
                     marker_color='#89b4fa', opacity=0.7,
                     hovertemplate='Intensité: %{x:.1f}<br>Count: %{y}<extra></extra>'),
        row=1, col=1
    )
    
    # 2. Arcsinh cofactor=5
    marker_arcsinh = DataTransformer.arcsinh_transform(marker_data, cofactor=5)
    fig.add_trace(
        go.Histogram(x=marker_arcsinh, nbinsx=200, name='Arcsinh (5)',
                     marker_color='#a6e3a1', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=1, col=2
    )
    
    # 3. Logicle ou Arcsinh cofactor=150
    if FLOWKIT_AVAILABLE:
        marker_logicle = DataTransformer.logicle_transform(marker_data)
        transform_name = 'Logicle'
    else:
        marker_logicle = DataTransformer.arcsinh_transform(marker_data, cofactor=150)
        transform_name = 'Arcsinh (150)'
    
    fig.add_trace(
        go.Histogram(x=marker_logicle, nbinsx=200, name=transform_name,
                     marker_color='#f9e2af', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=2, col=1
    )
    
    # 4. Log10
    marker_log = DataTransformer.log_transform(marker_data, base=10, min_val=1)
    fig.add_trace(
        go.Histogram(x=marker_log, nbinsx=200, name='Log10',
                     marker_color='#cba6f7', opacity=0.7,
                     hovertemplate='Intensité: %{x:.2f}<br>Count: %{y}<extra></extra>'),
        row=2, col=2
    )
    
    # Mise en page
    fig.update_xaxes(title_text="Intensité brute", row=1, col=1)
    fig.update_xaxes(title_text="Intensité transformée", row=1, col=2)
    fig.update_xaxes(title_text="Intensité transformée", row=2, col=1)
    fig.update_xaxes(title_text="Intensité log10", row=2, col=2)
    
    fig.update_yaxes(title_text="Fréquence", row=1, col=1)
    fig.update_yaxes(title_text="Fréquence", row=1, col=2)
    fig.update_yaxes(title_text="Fréquence", row=2, col=1)
    fig.update_yaxes(title_text="Fréquence", row=2, col=2)
    
    # Thème sombre et configuration
    fig.update_layout(
        title_text=f'Comparaison Transformations - {MARKER_TO_PLOT} ({len(marker_data):,} cellules)',
        title_font_size=16,
        height=900,
        showlegend=False,
        template='plotly_dark',
        hovermode='x unified'
    )
    
    # Afficher avec gestion d'erreur
    try:
        fig.show()
        print(f"\n✅ Visualisation interactive générée")
        print(f"   💡 Utilisez les outils Plotly: Zoom (box select), Pan, Reset, Download")
    except Exception as e:
        print(f"\n⚠️ Erreur affichage Plotly: {e}")
        print("   → Affichage en HTML dans le notebook...")
        
        # Alternative: Afficher le HTML directement dans le notebook
        try:
            from IPython.display import HTML, display
            html_str = fig.to_html(include_plotlyjs='cdn', include_mathjax='cdn')
            display(HTML(html_str))
            print(f"   ✅ Graphique affiché en HTML (pleinement interactif)")
        except Exception as e2:
            print(f"   ❌ Erreur HTML: {e2}")
            # Dernier recours: sauvegarder en fichier
            html_file = 'plotly_visualization.html'
            fig.write_html(html_file)
            print(f"   → Fichier sauvegardé: {html_file}")
            print(f"   → Ouvrez ce fichier dans votre navigateur pour l'interactivité complète")
    
    print(f"   Cellules: {len(marker_data):,}")
    print(f"   Min brut: {marker_data.min():.2f} | Max brut: {marker_data.max():.2f}")
else:
    # Fallback matplotlib si Plotly non disponible
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    ax = axes[0]
    ax.hist(marker_data, bins=200, color='#89b4fa', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Brut (Linéaire)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité brute')
    ax.set_ylabel('Fréquence')
    
    ax = axes[1]
    marker_arcsinh = DataTransformer.arcsinh_transform(marker_data, cofactor=5)
    ax.hist(marker_arcsinh, bins=200, color='#a6e3a1', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Arcsinh (cofactor=5)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée')
    ax.set_ylabel('Fréquence')
    
    ax = axes[2]
    if FLOWKIT_AVAILABLE:
        marker_logicle = DataTransformer.logicle_transform(marker_data)
        ax.hist(marker_logicle, bins=200, color='#f9e2af', alpha=0.7, edgecolor='none')
        ax.set_title(f'{MARKER_TO_PLOT} - Logicle', fontsize=12, fontweight='bold')
    else:
        marker_logicle = DataTransformer.arcsinh_transform(marker_data, cofactor=150)
        ax.hist(marker_logicle, bins=200, color='#f9e2af', alpha=0.7, edgecolor='none')
        ax.set_title(f'{MARKER_TO_PLOT} - Arcsinh (cofactor=150)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée')
    ax.set_ylabel('Fréquence')
    
    ax = axes[3]
    marker_log = DataTransformer.log_transform(marker_data, base=10, min_val=1)
    ax.hist(marker_log, bins=200, color='#cba6f7', alpha=0.7, edgecolor='none')
    ax.set_title(f'{MARKER_TO_PLOT} - Log10', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intensité transformée (log10)')
    ax.set_ylabel('Fréquence')
    
    plt.suptitle(f'Comparaison Transformations - {MARKER_TO_PLOT} ({len(marker_data):,} cellules)', 
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

## 5. Pre-Gating: Élimination des Débris et Doublets

Application du pre-gating automatique:
1. **gate_viable_cells()**: Exclusion des débris basé sur percentiles FSC/SSC
2. **gate_singlets()**: Exclusion des doublets via ratio FSC-A/FSC-H

In [ ]:
# =============================================================================
# APPLICATION DU PRE-GATING (OPTIONNEL)
# =============================================================================

# ACTIVER/DÉSACTIVER LE PRE-GATING
APPLY_PREGATING = False  # Mettre True pour appliquer le pre-gating, False pour le skip

# Paramètres de gating (utilisés si APPLY_PREGATING = True)
MIN_PERCENTILE = 2.0    # Exclusion débris (bas)
MAX_PERCENTILE = 98.0   # Exclusion doublets (haut)
RATIO_MIN = 0.6         # Ratio FSC-A/FSC-H minimum
RATIO_MAX = 1.5         # Ratio FSC-A/FSC-H maximum

# Données avant gating
X_raw = combined_data.X
if hasattr(X_raw, 'toarray'):
    X_raw = X_raw.toarray()
n_before = X_raw.shape[0]

if APPLY_PREGATING:
    print("🚪 PRE-GATING AUTOMATIQUE")
    print("="*60)
    print(f"   Percentiles FSC/SSC: [{MIN_PERCENTILE}%, {MAX_PERCENTILE}%]")
    print(f"   Ratio singlets: [{RATIO_MIN}, {RATIO_MAX}]")
    print(f"\n📊 Avant gating: {n_before:,} cellules")

    # Gate 1: Cellules viables (FSC/SSC)
    mask_viable = PreGating.gate_viable_cells(
        X_raw, var_names, 
        min_percentile=MIN_PERCENTILE, 
        max_percentile=MAX_PERCENTILE
    )
    n_after_viable = mask_viable.sum()
    print(f"   → Après gate viable: {n_after_viable:,} ({n_after_viable/n_before*100:.1f}%)")

    # Gate 2: Singlets (FSC-A/FSC-H ratio)
    mask_singlets = PreGating.gate_singlets(
        X_raw, var_names,
        ratio_min=RATIO_MIN,
        ratio_max=RATIO_MAX
    )
    n_after_singlets = mask_singlets.sum()
    print(f"   → Après gate singlets: {n_after_singlets:,} ({n_after_singlets/n_before*100:.1f}%)")

    # Masque combiné
    mask_final = mask_viable & mask_singlets
    n_final = mask_final.sum()
    n_excluded = n_before - n_final

    print(f"\n✅ Après pre-gating complet: {n_final:,} cellules")
    print(f"   → Exclus: {n_excluded:,} ({n_excluded/n_before*100:.1f}%)")
else:
    print("⏭️ PRE-GATING DÉSACTIVÉ")
    print("="*60)
    print(f"   → Toutes les {n_before:,} cellules seront conservées")
    
    # Masque qui garde tout
    mask_final = np.ones(n_before, dtype=bool)
    n_final = n_before

In [ ]:
# =============================================================================
# VISUALISATION AVANT/APRÈS GATING (si pre-gating activé)
# =============================================================================

if APPLY_PREGATING and fsc_col and ssc_col:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Sous-échantillonner
    n_sample = min(30000, n_before)
    idx_sample = np.random.choice(n_before, n_sample, replace=False)
    
    fsc_idx = var_names.index(fsc_col)
    ssc_idx = var_names.index(ssc_col)
    
    # Plot 1: Avant gating
    ax = axes[0]
    ax.hexbin(X_raw[idx_sample, fsc_idx], X_raw[idx_sample, ssc_idx],
              gridsize=80, cmap='Blues', mincnt=1)
    ax.set_xlabel(fsc_col)
    ax.set_ylabel(ssc_col)
    ax.set_title(f'AVANT Gating\n({n_before:,} cellules)', fontsize=11, fontweight='bold')
    
    # Plot 2: Cellules exclues
    ax = axes[1]
    excluded_mask = ~mask_final
    included_sample = mask_final[idx_sample]
    excluded_sample = ~included_sample
    
    ax.scatter(X_raw[idx_sample][excluded_sample, fsc_idx], 
               X_raw[idx_sample][excluded_sample, ssc_idx],
               s=1, c='#f38ba8', alpha=0.5, label='Exclus')
    ax.scatter(X_raw[idx_sample][included_sample, fsc_idx], 
               X_raw[idx_sample][included_sample, ssc_idx],
               s=1, c='#a6e3a1', alpha=0.5, label='Conservés')
    ax.set_xlabel(fsc_col)
    ax.set_ylabel(ssc_col)
    ax.set_title(f'Gating Overlay\n(🟢 conservés, 🔴 exclus)', fontsize=11, fontweight='bold')
    ax.legend(markerscale=10, loc='upper right')
    
    # Plot 3: Après gating
    ax = axes[2]
    X_gated = X_raw[mask_final]
    n_gated_sample = min(30000, len(X_gated))
    idx_gated = np.random.choice(len(X_gated), n_gated_sample, replace=False)
    
    ax.hexbin(X_gated[idx_gated, fsc_idx], X_gated[idx_gated, ssc_idx],
              gridsize=80, cmap='Greens', mincnt=1)
    ax.set_xlabel(fsc_col)
    ax.set_ylabel(ssc_col)
    ax.set_title(f'APRÈS Gating\n({n_final:,} cellules)', fontsize=11, fontweight='bold')
    
    plt.suptitle('Comparaison Pre-Gating FSC/SSC', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
elif not APPLY_PREGATING:
    print("ℹ️ Visualisation du gating non affichée (pre-gating désactivé)")

In [ ]:
# =============================================================================
# CRÉATION DU SECOND ANNDATA (avec ou sans gating)
# =============================================================================

# Créer l'AnnData filtré (ou copie complète si pas de gating)
combined_gated = combined_data[mask_final].copy()

if APPLY_PREGATING:
    print(f"✅ AnnData après gating: {combined_gated.shape}")
    print(f"   → {combined_gated.shape[0]:,} cellules conservées")
    print(f"   → {combined_gated.shape[1]} marqueurs")
else:
    print(f"✅ AnnData créé (sans pre-gating): {combined_gated.shape}")
    print(f"   → {combined_gated.shape[0]:,} cellules (toutes conservées)")
    print(f"   → {combined_gated.shape[1]} marqueurs")

## 6. Transformation des Données (Arcsinh / Logicle)

Les données brutes de cytométrie nécessitent une transformation pour:
- Gérer les valeurs négatives (compensation)
- Compresser la plage dynamique
- Améliorer la visualisation des populations faiblement exprimées

### Transformations disponibles:
- **Arcsinh (cofactor=5)**: Recommandé pour flow cytometry
- **Logicle**: Transformation biexponentielle (standard ISAC)
- **Log10**: Transformation logarithmique simple

In [ ]:
# CONFIGURATION DE LA TRANSFORMATION

# Choix de la transformation
TRANSFORM_TYPE = "arcsinh"  # Options: "arcsinh", "logicle", "log10", "none"
COFACTOR = 5  # Pour arcsinh: 5 (flow)

# Appliquer uniquement aux marqueurs de fluorescence (pas FSC/SSC/Time)
APPLY_TO_SCATTER = False

print("TRANSFORMATION DES DONNÉES")
print("="*60)
print(f"   Type: {TRANSFORM_TYPE.upper()}")
if TRANSFORM_TYPE == "arcsinh":
    print(f"   Cofacteur: {COFACTOR}")
print(f"   Appliquer au scatter: {'Oui' if APPLY_TO_SCATTER else 'Non'}")

In [ ]:
# APPLICATION DE LA TRANSFORMATION

# Extraire les données
X_gated = combined_gated.X
if hasattr(X_gated, 'toarray'):
    X_gated = X_gated.toarray()

# Copie pour transformation
X_transformed = X_gated.copy()

# Déterminer les indices des colonnes à transformer
if APPLY_TO_SCATTER:
    cols_to_transform = list(range(len(var_names)))
else:
    # Exclure FSC, SSC, Time
    scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']
    cols_to_transform = [i for i, name in enumerate(var_names) 
                         if not any(p in name.upper() for p in scatter_patterns)]

print(f"\nColonnes à transformer: {len(cols_to_transform)}/{len(var_names)}")

# Appliquer la transformation
if TRANSFORM_TYPE == "arcsinh":
    print(f"\n⚡ Application Arcsinh (cofactor={COFACTOR})...")
    X_transformed[:, cols_to_transform] = DataTransformer.arcsinh_transform(
        X_gated[:, cols_to_transform], cofactor=COFACTOR
    )
    
elif TRANSFORM_TYPE == "logicle":
    print("\n⚡ Application Logicle...")
    X_transformed[:, cols_to_transform] = DataTransformer.logicle_transform(
        X_gated[:, cols_to_transform]
    )
    
elif TRANSFORM_TYPE == "log10":
    print("\n⚡ Application Log10...")
    X_transformed[:, cols_to_transform] = DataTransformer.log_transform(
        X_gated[:, cols_to_transform]
    )
    
else:
    print("\n⚠️ Pas de transformation appliquée")

# Vérifier les résultats
print(f"\n✅ Transformation terminée!")
print(f"   Plage avant: [{X_gated[:, cols_to_transform].min():.2f}, {X_gated[:, cols_to_transform].max():.2f}]")
print(f"   Plage après: [{X_transformed[:, cols_to_transform].min():.2f}, {X_transformed[:, cols_to_transform].max():.2f}]")

## 7. Comparaison Avant/Après Transformation

Visualisation côte à côte des distributions pour valider l'effet de la transformation.

In [ ]:
# COMPARAISON DISTRIBUTIONS AVANT/APRÈS

# Sélectionner quelques marqueurs représentatifs
markers_compare = [var_names[i] for i in cols_to_transform[:6]]

fig, axes = plt.subplots(2, len(markers_compare), figsize=(4*len(markers_compare), 8))

for i, marker in enumerate(markers_compare):
    col_idx = var_names.index(marker)
    
    # Avant transformation
    ax = axes[0, i]
    data_before = X_gated[:, col_idx]
    ax.hist(data_before, bins=80, color='#f38ba8', alpha=0.7, edgecolor='none')
    ax.set_title(f'{marker}\n(Brut)', fontsize=10, fontweight='bold')
    ax.axvline(0, color='white', linestyle='--', alpha=0.5)
    if i == 0:
        ax.set_ylabel('AVANT\nCount', fontsize=11, fontweight='bold')
    
    # Après transformation
    ax = axes[1, i]
    data_after = X_transformed[:, col_idx]
    ax.hist(data_after, bins=80, color='#a6e3a1', alpha=0.7, edgecolor='none')
    ax.set_title(f'{TRANSFORM_TYPE.upper()}', fontsize=10)
    ax.axvline(0, color='white', linestyle='--', alpha=0.5)
    if i == 0:
        ax.set_ylabel('APRÈS\nCount', fontsize=11, fontweight='bold')

plt.suptitle(f'Comparaison des Distributions: Brut vs {TRANSFORM_TYPE.upper()} (cofactor={COFACTOR})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# TEST DE DIFFÉRENTS COFACTEURS (pour tuning)

print("COMPARAISON DES COFACTEURS ARCSINH")
print("="*60)

# Sélectionner un marqueur représentatif
test_marker = markers_compare[0]
test_idx = var_names.index(test_marker)
test_data = X_gated[:, test_idx]

# Tester différents cofacteurs
cofactors = [1, 5, 50, 150, 500]

fig, axes = plt.subplots(1, len(cofactors)+1, figsize=(4*(len(cofactors)+1), 4))

# Données brutes
ax = axes[0]
ax.hist(test_data, bins=80, color='#89b4fa', alpha=0.7, edgecolor='none')
ax.set_title('Brut\n(pas de transfo)', fontsize=10, fontweight='bold')
ax.set_xlabel(test_marker)

# Transformations avec différents cofacteurs
for i, cof in enumerate(cofactors):
    ax = axes[i+1]
    transformed = DataTransformer.arcsinh_transform(test_data, cofactor=cof)
    ax.hist(transformed, bins=80, color='#cba6f7', alpha=0.7, edgecolor='none')
    ax.set_title(f'Arcsinh\ncofactor={cof}', fontsize=10, fontweight='bold')
    ax.set_xlabel(test_marker)

plt.suptitle(f'🔧 Impact du Cofacteur sur la Distribution ({test_marker})', 
             fontsize=13, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

## 8. Préparation des Données pour FlowSOM

Sélection des colonnes pour le clustering:
- Exclusion des paramètres scatter (FSC, SSC) et Time
- Conservation uniquement des marqueurs de fluorescence
- Nettoyage final (NaN/Inf)

In [ ]:
# SÉLECTION DES COLONNES POUR FLOWSOM

# Option: exclure FSC/SSC/Time
EXCLUDE_SCATTER = True

# Identifier les colonnes à utiliser
scatter_patterns = ['FSC', 'SSC', 'TIME', 'EVENT']

if EXCLUDE_SCATTER:
    cols_to_use = [i for i, name in enumerate(var_names) 
                   if not any(p in name.upper() for p in scatter_patterns)]
else:
    cols_to_use = list(range(len(var_names)))

used_markers = [var_names[i] for i in cols_to_use]

print("COLONNES POUR FLOWSOM")
print("="*60)
print(f"   Exclure scatter: {'Oui' if EXCLUDE_SCATTER else 'Non'}")
print(f"   Colonnes sélectionnées: {len(cols_to_use)}/{len(var_names)}")
print(f"\nMarqueurs utilisés:")
for i, marker in enumerate(used_markers):
    print(f"   [{i:2d}] {marker}")

In [ ]:
# CRÉATION DE L'ANNDATA TRANSFORMÉ ET EXPLORATION POST-ARCSINH

# Créer un nouvel AnnData avec les données transformées (X_transformed)
import anndata as ad

# Créer adata_flowsom - le nouvel AnnData pour FlowSOM avec données transformées
adata_flowsom = ad.AnnData(
    X=X_transformed,  # Données POST-transformation arcsinh
    obs=combined_gated.obs.copy(),  # Copie des métadonnées
    var=combined_gated.var.copy() if combined_gated.var is not None else None
)

# Ajouter les noms de variables
adata_flowsom.var_names = var_names

print("="*70)
print("CRÉATION ANNDATA POUR FLOWSOM (DONNÉES POST-ARCSINH)")
print("="*70)
print(f"\n✅ Nouvel AnnData 'adata_flowsom' créé avec données transformées")
print(f"   Shape: {adata_flowsom.shape}")
print(f"   Observations (cellules): {adata_flowsom.n_obs:,}")
print(f"   Variables (marqueurs): {adata_flowsom.n_vars}")

# ============================================================================
# EXPLORATION DU DATAFRAME POST-TRANSFORMATION
# ============================================================================

# Extraire la matrice transformée depuis le NOUVEL AnnData
X_trans = adata_flowsom.X
if hasattr(X_trans, 'toarray'):
    X_trans = X_trans.toarray()

# Créer un DataFrame pour exploration
df_transformed = pd.DataFrame(X_trans, columns=var_names)
df_transformed['condition'] = adata_flowsom.obs['condition'].values
df_transformed['file_origin'] = adata_flowsom.obs['file_origin'].values

print("\n" + "="*70)
print("📋 APERÇU DES DONNÉES TRANSFORMÉES (premières 10 lignes)")
print("="*70)
print(f"Shape du DataFrame: {df_transformed.shape}")
display(df_transformed.head(10))

# VÉRIFICATION DES NaN ET Inf POST-ARCSINH

print("\n" + "="*70)
print("🔍 VÉRIFICATION DES VALEURS NaN ET Inf POST-ARCSINH")
print("="*70)

# Colonnes numériques uniquement
numeric_cols = df_transformed.select_dtypes(include=[np.number]).columns.tolist()

# Comptage des NaN
nan_counts = df_transformed[numeric_cols].isna().sum()
total_nan = nan_counts.sum()

# Comptage des Inf (positifs et négatifs)
inf_pos_counts = (df_transformed[numeric_cols] == np.inf).sum()
inf_neg_counts = (df_transformed[numeric_cols] == -np.inf).sum()
total_inf_pos = inf_pos_counts.sum()
total_inf_neg = inf_neg_counts.sum()
total_inf = total_inf_pos + total_inf_neg

total_cells = df_transformed.shape[0] * len(numeric_cols)

print(f"\n📈 RÉSUMÉ GLOBAL:")
print(f"   Total valeurs analysées: {total_cells:,}")
print(f"   Total NaN:    {total_nan:,} ({100*total_nan/total_cells:.4f}%)")
print(f"   Total +Inf:   {total_inf_pos:,} ({100*total_inf_pos/total_cells:.4f}%)")
print(f"   Total -Inf:   {total_inf_neg:,} ({100*total_inf_neg/total_cells:.4f}%)")

# Détail par colonne si problèmes détectés
if total_nan > 0 or total_inf > 0:
    print(f"\n⚠️ DÉTAIL PAR COLONNE AVEC PROBLÈMES:")
    print("-"*60)
    for col in numeric_cols:
        n_nan = df_transformed[col].isna().sum()
        n_inf_pos = (df_transformed[col] == np.inf).sum()
        n_inf_neg = (df_transformed[col] == -np.inf).sum()
        if n_nan > 0 or n_inf_pos > 0 or n_inf_neg > 0:
            print(f"   {col:30s}: NaN={n_nan:,}, +Inf={n_inf_pos:,}, -Inf={n_inf_neg:,}")
else:
    print(f"\n✅ Aucune valeur NaN ou Inf détectée - Données propres!")

# ============================================================================
# STATISTIQUES DESCRIPTIVES POST-ARCSINH
# ============================================================================
print("\n" + "="*70)
print("📊 STATISTIQUES DESCRIPTIVES POST-ARCSINH")
print("="*70)
display(df_transformed[numeric_cols].describe())

# ============================================================================
# VÉRIFICATION DES RANGES POST-TRANSFORMATION
# ============================================================================
print("\n" + "="*70)
print("📏 VÉRIFICATION DES RANGES POST-TRANSFORMATION")
print("="*70)
print("(arcsinh avec cofactor=150 donne typiquement des valeurs entre -5 et 10)\n")

for col in used_markers[:10]:  # Premiers 10 marqueurs utilisés
    col_min = df_transformed[col].min()
    col_max = df_transformed[col].max()
    col_mean = df_transformed[col].mean()
    print(f"   {col:30s}: min={col_min:8.3f}, max={col_max:8.3f}, mean={col_mean:8.3f}")

In [ ]:
# NETTOYAGE FINAL ET VALIDATION DE L'ANNDATA POUR FLOWSOM

# Nettoyage final: remplacer NaN/Inf par 0 dans adata_flowsom
X_final = adata_flowsom.X
if hasattr(X_final, 'toarray'):
    X_final = X_final.toarray()

# Vérifier et nettoyer
nan_mask = ~np.isfinite(X_final)
n_nan = nan_mask.sum()
if n_nan > 0:
    print(f"⚠️ {n_nan} valeurs NaN/Inf détectées et remplacées par 0")
    X_final = np.nan_to_num(X_final, nan=0.0, posinf=0.0, neginf=0.0)
    adata_flowsom.X = X_final
else:
    print("✅ Aucune valeur problématique - pas de nettoyage nécessaire")

print(f"\n✅ AnnData 'adata_flowsom' prêt pour FlowSOM:")
print(f"   Shape: {adata_flowsom.shape}")
print(f"   Colonnes pour clustering: {len(cols_to_use)}")

# Résumé par condition
print(f"\n📊 Distribution par condition:")
for condition in adata_flowsom.obs['condition'].unique():
    n = (adata_flowsom.obs['condition'] == condition).sum()
    print(f"   {condition}: {n:,} cellules")

## 9. Exécution du Clustering FlowSOM

Configuration et lancement de l'analyse FlowSOM avec:
- Grille SOM (xdim × ydim)
- Nombre de métaclusters
- Seed pour reproductibilité

In [ ]:
# PARAMÈTRES FLOWSOM

# Dimensions de la grille SOM
XDIM = 10
YDIM = 10

# Nombre de métaclusters
N_CLUSTERS = 2

# Seed pour reproductibilité
SEED = 42

# Auto-clustering avec silhouette score?
AUTO_CLUSTER = False
MAX_CLUSTERS_AUTO = 20

print("PARAMÈTRES FLOWSOM")
print("="*60)
print(f"   Grille SOM: {XDIM} × {YDIM} = {XDIM*YDIM} nodes")
print(f"   Métaclusters: {N_CLUSTERS}")
print(f"   Seed: {SEED}")
print(f"   Auto-clustering: {'Oui' if AUTO_CLUSTER else 'Non'}")

In [ ]:
# FONCTION POUR TROUVER LE NOMBRE OPTIMAL DE CLUSTERS (optionnel)
# ⚠️ Le silhouette score nécessite une matrice N×N → impossible avec 1M cellules
# Solution: Sous-échantillonner pour l'évaluation silhouette uniquement

SAMPLE_SIZE_SILHOUETTE = 10000  # Taille de l'échantillon pour silhouette

def find_optimal_clusters(data, cols_to_use, seed, max_clusters=20, sample_size=10000):
    """
    Trouve le nombre optimal de métaclusters via silhouette score.
    Utilise un échantillon représentatif pour éviter l'explosion mémoire.
    """
    print("Recherche du nombre optimal de clusters...")
    
    np.random.seed(seed)
    
    X = data.X
    if hasattr(X, 'toarray'):
        X = X.toarray()
    
    X_full = X[:, cols_to_use]
    X_full = np.nan_to_num(X_full, nan=0.0)
    
    n_total = X_full.shape[0]
    
    # Sous-échantillonner pour silhouette (sinon O(N²) mémoire)
    if n_total > sample_size:
        print(f"   ⚠️ {n_total:,} cellules → échantillon de {sample_size:,} pour silhouette")
        idx = np.random.choice(n_total, sample_size, replace=False)
        X_sample = X_full[idx]
    else:
        print(f"   Utilisation de {n_total:,} cellules")
        X_sample = X_full
    
    scores = []
    cluster_range = range(2, min(max_clusters + 1, len(X_sample) // 10))
    
    for k in cluster_range:
        try:
            clustering = AgglomerativeClustering(n_clusters=k)
            labels = clustering.fit_predict(X_sample)
            score = silhouette_score(X_sample, labels)
            scores.append((k, score))
            print(f"   k={k}: silhouette={score:.4f}")
        except Exception as e:
            print(f"   k={k}: erreur - {e}")
    
    if scores:
        best_k, best_score = max(scores, key=lambda x: x[1])
        print(f"\n✅ Nombre optimal: {best_k} (silhouette={best_score:.4f})")
        return best_k
    
    return 10  # Valeur par défaut

# Exécuter si AUTO_CLUSTER est activé
if AUTO_CLUSTER:
    N_CLUSTERS = find_optimal_clusters(
        combined_gated, cols_to_use, SEED, 
        MAX_CLUSTERS_AUTO, SAMPLE_SIZE_SILHOUETTE
    )
    print(f"\n📊 Utilisation de {N_CLUSTERS} métaclusters")

In [ ]:
# =============================================================================
# EXÉCUTION FLOWSOM
# =============================================================================

import time
start_time = time.time()

# Exécuter FlowSOM avec adata_flowsom (données transformées arcsinh)
fsom = fs.FlowSOM(
    adata_flowsom,  # ← IMPORTANT: utilise les données POST-transformation
    cols_to_use=cols_to_use,
    xdim=XDIM,
    ydim=YDIM,
    n_clusters=N_CLUSTERS,
    seed=SEED
)

elapsed = time.time() - start_time
print(f"\nTemps d'exécution: {elapsed:.2f} secondes")

# Récupérer les données de clustering
cell_data = fsom.get_cell_data()
cluster_data = fsom.get_cluster_data()

# Ajouter les métadonnées originales
cell_data.obs['condition'] = adata_flowsom.obs['condition'].values
cell_data.obs['file_origin'] = adata_flowsom.obs['file_origin'].values

print(f"\n✅ FlowSOM terminé!")
print(f"   Cellules analysées: {cell_data.shape[0]:,}")
print(f"   Nodes SOM: {cluster_data.shape[0]}")
print(f"   Métaclusters: {N_CLUSTERS}")

## 10. Visualisation des Résultats FlowSOM

Génération des visualisations standards:
- Heatmap d'expression par métacluster
- Arbre MST (Minimum Spanning Tree)
- Star Charts
- Distribution par condition

In [ ]:
# =============================================================================
# HEATMAP D'EXPRESSION PAR MÉTACLUSTER
# =============================================================================

print("📊 Génération de la Heatmap d'expression...")

# Récupérer les données
X = cell_data.X
if hasattr(X, 'toarray'):
    X = X.toarray()

metaclustering = cell_data.obs['metaclustering'].values

# Calculer la MFI (Mean Fluorescence Intensity) par métacluster
mfi_matrix = np.zeros((N_CLUSTERS, len(cols_to_use)))

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    if mask.sum() > 0:
        mfi_matrix[i, :] = np.nanmean(X[mask][:, cols_to_use], axis=0)

# Normalisation Z-score pour la heatmap
mfi_normalized = (mfi_matrix - np.nanmean(mfi_matrix, axis=0)) / (np.nanstd(mfi_matrix, axis=0) + 1e-10)

# Créer la heatmap
fig, ax = plt.subplots(figsize=(14, 8))

im = ax.imshow(mfi_normalized.T, aspect='auto', cmap='RdBu_r', vmin=-2, vmax=2)

# Labels
ax.set_yticks(range(len(used_markers)))
ax.set_yticklabels(used_markers, fontsize=9)
ax.set_xticks(range(N_CLUSTERS))
ax.set_xticklabels([f'MC{i}' for i in range(N_CLUSTERS)], fontsize=10)

ax.set_title('Heatmap - Expression par Métacluster (Z-score)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Métacluster', fontsize=12)
ax.set_ylabel('Marqueur', fontsize=12)

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8, label='Z-score')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# STAR CHART FLOWSOM (MST View)
# =============================================================================

print("Génération du Star Chart MST...")

try:
    # Utiliser l'API FlowSOM pour le Star Chart
    fig_stars = fs.pl.plot_stars(
        fsom,
        background_values=fsom.get_cluster_data().obs.metaclustering,
        view="MST"
    )
    plt.suptitle('FlowSOM Star Chart (MST View)', fontsize=14, fontweight='bold')
    plt.show()
except Exception as e:
    print(f"Erreur Star Chart: {e}")
    print("   Utilisation de la visualisation alternative...")

In [ ]:
# =============================================================================
# VISUALISATION GRILLE SOM (xGrid, yGrid) - Style FlowSOM R exact
# =============================================================================

print("📊 VISUALISATION GRILLE SOM (style FlowSOM R avec cercles)")
print("="*70)

# =====================================================================
# FONCTION JITTER CIRCULAIRE (style FlowSOM R)
# =====================================================================
def circular_jitter_viz(n_points, cluster_ids, node_sizes, max_radius=0.45, min_radius=0.1):
    """
    Génère un jitter circulaire style FlowSOM R.
    Le rayon des cercles dépend du nombre de cellules dans le node.
    """
    theta = np.random.uniform(0, 2 * np.pi, n_points)
    u = np.random.uniform(0, 1, n_points)
    
    max_size_val = node_sizes.max()
    
    radii = np.zeros(n_points, dtype=np.float32)
    for i in range(n_points):
        node_id = int(cluster_ids[i])
        node_size = node_sizes[node_id]
        size_ratio = np.sqrt(node_size / max_size_val)
        node_radius = min_radius + (max_radius - min_radius) * size_ratio
        radii[i] = node_radius
    
    r = np.sqrt(u) * radii
    
    jitter_x = r * np.cos(theta)
    jitter_y = r * np.sin(theta)
    
    return jitter_x.astype(np.float32), jitter_y.astype(np.float32)

try:
    # Récupérer les coordonnées de grille
    grid_coords = cluster_data.obsm.get('grid', None)
    
    if grid_coords is not None:
        # Récupérer les infos de clustering
        clustering = cell_data.obs['clustering'].values
        metaclustering_nodes = cluster_data.obs['metaclustering'].values
        conditions = cell_data.obs['condition'].values
        
        # Calculer les coordonnées de grille pour chaque cellule
        xGrid_base = np.array([grid_coords[int(c), 0] for c in clustering], dtype=np.float32)
        yGrid_base = np.array([grid_coords[int(c), 1] for c in clustering], dtype=np.float32)
        
        # Décaler pour commencer à 1
        xGrid_shifted = xGrid_base - xGrid_base.min() + 1
        yGrid_shifted = yGrid_base - yGrid_base.min() + 1
        
        # Métacluster pour chaque cellule
        metaclustering_cells = np.array([metaclustering_nodes[int(c)] for c in clustering])
        
        # Calculer la taille de chaque node
        n_nodes = len(cluster_data)
        node_sizes = np.zeros(n_nodes, dtype=np.float32)
        for i in range(n_nodes):
            node_sizes[i] = (clustering == i).sum()
        
        # JITTER CIRCULAIRE style FlowSOM R
        MAX_NODE_SIZE = 0.45
        MIN_NODE_SIZE = 0.1
        np.random.seed(SEED)
        jitter_x, jitter_y = circular_jitter_viz(len(clustering), clustering, node_sizes, 
                                                  max_radius=MAX_NODE_SIZE, 
                                                  min_radius=MIN_NODE_SIZE)
        
        print(f"🎯 Jitter circulaire appliqué (rayon proportionnel à la taille du node)")
        print(f"   Rayon min: {MIN_NODE_SIZE}, Rayon max: {MAX_NODE_SIZE}")
        
        # Créer la figure avec 2 sous-plots
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        
        # =====================================================================
        # Plot 1: Grille SOM colorée par Métacluster
        # =====================================================================
        ax1 = axes[0]
        
        n_meta = len(np.unique(metaclustering_nodes))
        cmap = plt.cm.tab20 if n_meta <= 20 else plt.cm.turbo
        
        scatter1 = ax1.scatter(
            xGrid_shifted + jitter_x, 
            yGrid_shifted + jitter_y,
            c=metaclustering_cells,
            cmap=cmap,
            s=5,
            alpha=0.5,
            edgecolors='none'
        )
        
        # Ajouter les labels des métaclusters au centre de chaque node
        for node_id in range(n_nodes):
            if node_sizes[node_id] > 0:
                x_pos = grid_coords[node_id, 0] - xGrid_base.min() + 1
                y_pos = grid_coords[node_id, 1] - yGrid_base.min() + 1
                meta_id = metaclustering_nodes[node_id]
                ax1.annotate(
                    str(int(meta_id + 1)),
                    (x_pos, y_pos),
                    ha='center', va='center',
                    fontsize=8, fontweight='bold',
                    color='white',
                    bbox=dict(boxstyle='circle,pad=0.2', facecolor=cmap(meta_id / max(n_meta - 1, 1)), edgecolor='white', alpha=0.9)
                )
        
        ax1.set_xlabel('xGrid', fontsize=12, fontweight='bold')
        ax1.set_ylabel('yGrid', fontsize=12, fontweight='bold')
        ax1.set_title(f'Grille FlowSOM - {XDIM}x{YDIM} nodes\nColoré par Métacluster (style FlowSOM R)', 
                     fontsize=12, fontweight='bold')
        ax1.set_xlim(0.5, XDIM + 1.5)
        ax1.set_ylim(0.5, YDIM + 1.5)
        ax1.set_aspect('equal')
        ax1.grid(True, alpha=0.3, linestyle='--')
        
        cbar1 = plt.colorbar(scatter1, ax=ax1, label='Métacluster')
        
        # =====================================================================
        # Plot 2: Grille SOM colorée par Condition
        # =====================================================================
        ax2 = axes[1]
        
        condition_num = np.array([0 if c == 'Sain' else 1 for c in conditions])
        
        from matplotlib.colors import ListedColormap
        cmap_cond = ListedColormap(['#a6e3a1', '#f38ba8'])
        
        scatter2 = ax2.scatter(
            xGrid_shifted + jitter_x, 
            yGrid_shifted + jitter_y,
            c=condition_num,
            cmap=cmap_cond,
            s=5,
            alpha=0.5,
            edgecolors='none'
        )
        
        ax2.set_xlabel('xGrid', fontsize=12, fontweight='bold')
        ax2.set_ylabel('yGrid', fontsize=12, fontweight='bold')
        ax2.set_title(f'Grille FlowSOM - {XDIM}x{YDIM} nodes\nColoré par Condition (style FlowSOM R)', 
                     fontsize=12, fontweight='bold')
        ax2.set_xlim(0.5, XDIM + 1.5)
        ax2.set_ylim(0.5, YDIM + 1.5)
        ax2.set_aspect('equal')
        ax2.grid(True, alpha=0.3, linestyle='--')
        
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='#a6e3a1', edgecolor='white', label='Sain (NBM)'),
            Patch(facecolor='#f38ba8', edgecolor='white', label='Pathologique')
        ]
        ax2.legend(handles=legend_elements, loc='upper right')
        
        plt.tight_layout()
        plt.show()
        
        # Afficher les statistiques
        print(f"\n📋 STATISTIQUES DE LA GRILLE SOM:")
        print(f"   Dimensions: {XDIM} x {YDIM} = {XDIM * YDIM} nodes")
        print(f"   Nodes utilisés: {(node_sizes > 0).sum()} / {n_nodes}")
        print(f"   xGrid range: [{xGrid_shifted.min():.1f}, {xGrid_shifted.max():.1f}]")
        print(f"   yGrid range: [{yGrid_shifted.min():.1f}, {yGrid_shifted.max():.1f}]")
        
        # Afficher la taille des nodes
        print(f"\n📊 Distribution des tailles de nodes:")
        print(f"   Min: {node_sizes.min():.0f} cellules")
        print(f"   Max: {node_sizes.max():.0f} cellules")
        print(f"   Moyenne: {node_sizes.mean():.0f} cellules")
        
    else:
        print("⚠️ Coordonnées de grille non disponibles dans cluster_data.obsm['grid']")
        
except Exception as e:
    import traceback
    print(f"⚠️ Erreur visualisation grille: {e}")
    traceback.print_exc()

In [ ]:
# =============================================================================
# ARBRE MST EN MATPLOTLIB (alternative)
# =============================================================================

print("Génération de l'arbre MST...")

try:
    # Récupérer les coordonnées du layout MST
    layout = cluster_data.obsm.get('layout', None)
    
    if layout is not None:
        # Clustering et metaclustering
        clustering = cell_data.obs['clustering'].values
        metaclustering_nodes = cluster_data.obs['metaclustering'].values
        
        # Calculer la taille de chaque node
        n_nodes = len(cluster_data)
        node_sizes = np.zeros(n_nodes)
        for i in range(n_nodes):
            node_sizes[i] = (clustering == i).sum()
        
        # Normaliser les tailles
        max_size = node_sizes.max() if node_sizes.max() > 0 else 1
        sizes = 100 + (node_sizes / max_size) * 800
        
        # Couleurs par métacluster
        n_meta = len(np.unique(metaclustering_nodes))
        cmap = plt.cm.tab20 if n_meta <= 20 else plt.cm.turbo
        colors = [cmap(int(m) / max(n_meta - 1, 1)) for m in metaclustering_nodes]
        
        fig, ax = plt.subplots(figsize=(12, 10))
        
        # Scatter des nodes
        scatter = ax.scatter(layout[:, 0], layout[:, 1], 
                           s=sizes, c=colors, edgecolors='white', 
                           linewidths=1.5, alpha=0.9, zorder=2)
        
        # Annoter avec les numéros
        for i in range(n_nodes):
            ax.annotate(str(int(metaclustering_nodes[i])), 
                       (layout[i, 0], layout[i, 1]),
                       ha='center', va='center', fontsize=8, 
                       color='white', fontweight='bold')
        
        ax.set_xlabel('xNodes', fontsize=12, fontweight='bold')
        ax.set_ylabel('yNodes', fontsize=12, fontweight='bold')
        ax.set_title(f'Arbre MST - {n_nodes} nodes, {n_meta} métaclusters', 
                    fontsize=14, fontweight='bold', pad=15)
        ax.grid(True, alpha=0.15, linestyle='--')
        
        # Légende
        from matplotlib.patches import Patch
        if n_meta <= 15:
            legend_elements = [Patch(facecolor=cmap(i/max(n_meta-1, 1)), 
                                    label=f'MC {i}') for i in range(n_meta)]
            ax.legend(handles=legend_elements, loc='center left', 
                     bbox_to_anchor=(1.02, 0.5), fontsize=9)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Layout MST non disponible")
        
except Exception as e:
    print(f"⚠️ Erreur MST: {e}")

In [ ]:
# =============================================================================
# DISTRIBUTION PAR CONDITION (Sain vs Pathologique)
# =============================================================================

print("Distribution des métaclusters par condition...")

metaclustering = cell_data.obs['metaclustering'].values
conditions = cell_data.obs['condition'].values

healthy_pcts = []
patho_pcts = []

for i in range(N_CLUSTERS):
    mask_cluster = metaclustering == i
    
    # Pourcentage dans Sain
    mask_healthy = (conditions == 'Sain') & mask_cluster
    total_healthy = (conditions == 'Sain').sum()
    healthy_pcts.append((mask_healthy.sum() / total_healthy * 100) if total_healthy > 0 else 0)
    
    # Pourcentage dans Pathologique
    mask_patho = (conditions == 'Pathologique') & mask_cluster
    total_patho = (conditions == 'Pathologique').sum()
    patho_pcts.append((mask_patho.sum() / total_patho * 100) if total_patho > 0 else 0)

# Créer le graphique
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(N_CLUSTERS)
width = 0.35

bars1 = ax.bar(x - width/2, healthy_pcts, width, label='Sain (NBM)', 
               color='#a6e3a1', edgecolor='white', linewidth=0.5)
bars2 = ax.bar(x + width/2, patho_pcts, width, label='Pathologique', 
               color='#f38ba8', edgecolor='white', linewidth=0.5)

ax.set_xlabel('Métacluster', fontsize=12)
ax.set_ylabel('Pourcentage (%)', fontsize=12)
ax.set_title('Distribution des Métaclusters par Condition', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'MC{i}' for i in range(N_CLUSTERS)], fontsize=10)
ax.legend()
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Ajouter les valeurs sur les barres
for bar in bars1 + bars2:
    height = bar.get_height()
    if height > 1:  # N'afficher que si > 1%
        ax.annotate(f'{height:.1f}%',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3), textcoords="offset points",
                   ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Tableau récapitulatif
print("\nTableau récapitulatif:")
print("-" * 50)
print(f"{'MC':>4} | {'Sain (%)':>10} | {'Patho (%)':>10} | {'Diff':>8}")
print("-" * 50)
for i in range(N_CLUSTERS):
    diff = patho_pcts[i] - healthy_pcts[i]
    print(f"{i:>4} | {healthy_pcts[i]:>10.2f} | {patho_pcts[i]:>10.2f} | {diff:>+8.2f}")
print("-" * 50)

## 11. Analyse Détaillée des Métaclusters

Statistiques approfondies par métacluster:
- Nombre de cellules
- MFI par marqueur
- Phénotype caractéristique

In [ ]:
# =============================================================================
# STATISTIQUES PAR MÉTACLUSTER
# =============================================================================

print("📊 STATISTIQUES PAR MÉTACLUSTER")
print("="*80)

# Créer un DataFrame de statistiques
stats_data = []

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    n_cells = mask.sum()
    pct_total = n_cells / len(metaclustering) * 100
    
    # Calculer MFI pour chaque marqueur
    mfi = np.nanmean(X[mask][:, cols_to_use], axis=0) if n_cells > 0 else np.zeros(len(cols_to_use))
    
    # Top 3 marqueurs les plus exprimés
    top_indices = np.argsort(mfi)[::-1][:3]
    top_markers = [used_markers[idx] for idx in top_indices]
    
    stats_data.append({
        'Metacluster': i,
        'N_Cells': n_cells,
        'Pct_Total': pct_total,
        'Top_Markers': ', '.join(top_markers)
    })

df_stats = pd.DataFrame(stats_data)
print(df_stats.to_string(index=False))

# Graphique camembert
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart des tailles
ax = axes[0]
sizes = [s['N_Cells'] for s in stats_data]
labels = [f"MC{s['Metacluster']}" for s in stats_data]
colors = plt.cm.tab20(np.linspace(0, 1, N_CLUSTERS))

wedges, texts, autotexts = ax.pie(sizes, labels=labels, colors=colors, 
                                   autopct='%1.1f%%', pctdistance=0.8)
ax.set_title('Distribution des Cellules par Métacluster', fontsize=12, fontweight='bold')

# Bar chart des tailles
ax = axes[1]
ax.barh(range(N_CLUSTERS), sizes, color=colors, edgecolor='white')
ax.set_yticks(range(N_CLUSTERS))
ax.set_yticklabels(labels)
ax.set_xlabel('Nombre de cellules')
ax.set_title('Taille des Métaclusters', fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# PROFIL D'EXPRESSION DÉTAILLÉ PAR MÉTACLUSTER
# =============================================================================

print("\n📈 PROFIL D'EXPRESSION MOYEN PAR MÉTACLUSTER")
print("="*80)

# Créer un DataFrame avec MFI par marqueur et métacluster
mfi_matrix = np.zeros((N_CLUSTERS, len(used_markers)))

for i in range(N_CLUSTERS):
    mask = metaclustering == i
    if mask.sum() > 0:
        mfi_matrix[i] = np.nanmean(X[mask][:, cols_to_use], axis=0)

df_mfi = pd.DataFrame(mfi_matrix, 
                       columns=used_markers,
                       index=[f'MC{i}' for i in range(N_CLUSTERS)])

# Afficher le tableau formaté
print(df_mfi.round(2).to_string())

# Visualisation Radar/Spider plot pour les clusters principaux
from math import pi

# Sélectionner les 5 plus gros clusters
top_clusters = df_stats.nlargest(5, 'N_Cells')['Metacluster'].values

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

angles = [n / float(len(used_markers)) * 2 * pi for n in range(len(used_markers))]
angles += angles[:1]  # Fermer le polygone

colors = plt.cm.Set2(np.linspace(0, 1, len(top_clusters)))

for idx, cluster_id in enumerate(top_clusters):
    values = mfi_matrix[cluster_id].tolist()
    # Normaliser entre 0 et 1 pour la visualisation
    values_norm = (values - np.min(values)) / (np.max(values) - np.min(values) + 1e-10)
    values_norm = values_norm.tolist()
    values_norm += values_norm[:1]
    
    ax.plot(angles, values_norm, 'o-', linewidth=2, label=f'MC{cluster_id}', color=colors[idx])
    ax.fill(angles, values_norm, alpha=0.1, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(used_markers, size=9)
ax.set_title('Profil d\'Expression Normalisé des 5 Plus Gros Métaclusters', 
             fontsize=12, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

## 12. Export des Résultats

Sauvegarde des résultats d'analyse:
- **CSV**: Tableau avec métaclusters assignés à chaque cellule
- **FCS**: Fichier FCS avec colonne métacluster ajoutée

In [ ]:
# =============================================================================
# EXPORT CSV/FCS AVEC COORDONNÉES SOM (style FlowSOM R EXACT)
# =============================================================================

import os
from datetime import datetime

# Créer le dossier de sortie
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📊 PRÉPARATION DES DONNÉES POUR EXPORT (style FlowSOM R EXACT)")
print("="*70)

# =====================================================================
# PARAMÈTRES DE JITTER - STYLE FLOWSOM R EXACT
# Dans FlowSOM R, le jitter est CIRCULAIRE (pas carré!)
# La taille du cercle dépend du nombre de cellules dans le cluster
# Formule R: rnorm() * scale_factor * sqrt(node_size/max_size)
# =====================================================================
np.random.seed(SEED)  # Pour reproductibilité

# Paramètres FlowSOM R
MAX_NODE_SIZE = 0.45  # Rayon maximum du cercle (quand le node est le plus grand)
MIN_NODE_SIZE = 0.1   # Rayon minimum du cercle (pour éviter que les petits nodes disparaissent)

# Récupérer les coordonnées de grille et MST depuis cluster_data
grid_coords = cluster_data.obsm.get('grid', None)
layout_coords = cluster_data.obsm.get('layout', None)

# Récupérer le clustering pour mapper les coordonnées sur chaque cellule
clustering = cell_data.obs['clustering'].values
n_cells = len(clustering)
n_nodes = len(cluster_data)

# Calculer la taille de chaque node (nombre de cellules)
node_sizes = np.zeros(n_nodes, dtype=np.float32)
for i in range(n_nodes):
    node_sizes[i] = (clustering == i).sum()

max_size = node_sizes.max()
print(f"\n📊 Taille des nodes:")
print(f"   Min: {node_sizes.min():.0f} cellules")
print(f"   Max: {max_size:.0f} cellules")
print(f"   Total: {n_cells} cellules")

# Créer un DataFrame avec toutes les données
df_export = pd.DataFrame(X, columns=var_names)

# MetaCluster avec +1 pour Kaluza (éviter le 0, commencer à 1)
df_export['FlowSOM_metacluster'] = metaclustering + 1

# FlowSOM cluster (nodes) avec +1
df_export['FlowSOM_cluster'] = clustering + 1

# Ajouter les métadonnées si disponibles
if 'condition' in cell_data.obs.columns:
    df_export['Condition'] = cell_data.obs['condition'].values
    df_export['Condition_Num'] = np.where(df_export['Condition'] == 'Sain', 1, 2)
if 'file_origin' in cell_data.obs.columns:
    df_export['File_Origin'] = cell_data.obs['file_origin'].values

# =====================================================================
# FONCTION JITTER CIRCULAIRE (style FlowSOM R exact)
# Génère des points distribués uniformément dans un disque
# Le rayon dépend de la taille du cluster
# =====================================================================
def circular_jitter(n_points, cluster_ids, node_sizes, max_radius=0.45, min_radius=0.1):
    """
    Génère un jitter circulaire style FlowSOM R.
    
    Dans FlowSOM R, les cellules sont distribuées dans des cercles
    dont le rayon dépend du nombre de cellules dans le node.
    Plus un node a de cellules, plus le cercle est grand.
    
    Méthode: 
    - Angle theta uniforme [0, 2*pi]
    - Rayon r = sqrt(u) * max_r (pour distribution uniforme dans le disque)
    - Le max_r dépend de la taille du node
    """
    # Angle uniforme autour du cercle
    theta = np.random.uniform(0, 2 * np.pi, n_points)
    
    # Rayon - distribution uniforme dans le disque (sqrt pour uniformité)
    u = np.random.uniform(0, 1, n_points)
    
    # Calculer le rayon pour chaque cellule selon la taille de son cluster
    # Dans FlowSOM R, le rayon est proportionnel à sqrt(node_size/max_size)
    max_size_val = node_sizes.max()
    
    # Rayon pour chaque cellule
    radii = np.zeros(n_points, dtype=np.float32)
    for i in range(n_points):
        node_id = int(cluster_ids[i])
        node_size = node_sizes[node_id]
        # Rayon proportionnel à sqrt(taille relative)
        size_ratio = np.sqrt(node_size / max_size_val)
        # Interpolation entre min et max radius
        node_radius = min_radius + (max_radius - min_radius) * size_ratio
        radii[i] = node_radius
    
    # Rayon final pour distribution uniforme dans le disque
    r = np.sqrt(u) * radii
    
    # Convertir en coordonnées cartésiennes
    jitter_x = r * np.cos(theta)
    jitter_y = r * np.sin(theta)
    
    return jitter_x.astype(np.float32), jitter_y.astype(np.float32)

# =====================================================================
# COORDONNÉES GRILLE SOM (xGrid, yGrid) - Style FlowSOM R
# =====================================================================
print(f"\n🎯 Application du jitter CIRCULAIRE style FlowSOM R")
print(f"   Rayon min: {MIN_NODE_SIZE}, Rayon max: {MAX_NODE_SIZE}")

if grid_coords is not None:
    # Générer jitter CIRCULAIRE dépendant de la taille du node
    jitter_x, jitter_y = circular_jitter(n_cells, clustering, node_sizes, 
                                          max_radius=MAX_NODE_SIZE, 
                                          min_radius=MIN_NODE_SIZE)
    
    # Mapper les coordonnées de grille sur chaque cellule
    xGrid_base = np.array([grid_coords[int(c), 0] for c in clustering], dtype=np.float32)
    yGrid_base = np.array([grid_coords[int(c), 1] for c in clustering], dtype=np.float32)
    
    # Appliquer le jitter circulaire
    xGrid_jittered = xGrid_base + jitter_x
    yGrid_jittered = yGrid_base + jitter_y
    
    # Décaler pour que les axes commencent à 1 (X ET Y)
    xGrid = xGrid_jittered - xGrid_jittered.min() + 1.0
    yGrid = yGrid_jittered - yGrid_jittered.min() + 1.0
    
    df_export['xGrid'] = xGrid.astype(np.float32)
    df_export['yGrid'] = yGrid.astype(np.float32)
    
    print(f"✅ xGrid: [{xGrid.min():.3f} - {xGrid.max():.3f}]")
    print(f"✅ yGrid: [{yGrid.min():.3f} - {yGrid.max():.3f}]")

# =====================================================================
# COORDONNÉES MST (xNodes, yNodes) - Style FlowSOM R
# =====================================================================
if layout_coords is not None:
    # Mapper les coordonnées MST sur chaque cellule
    xNodes_base = np.array([layout_coords[int(c), 0] for c in clustering], dtype=np.float32)
    yNodes_base = np.array([layout_coords[int(c), 1] for c in clustering], dtype=np.float32)
    
    # Calculer l'échelle pour le jitter MST (proportionnel à l'espacement moyen)
    x_range = xNodes_base.max() - xNodes_base.min()
    y_range = yNodes_base.max() - yNodes_base.min()
    mst_scale = min(x_range, y_range) / (XDIM * 2)  # Proportionnel à la grille
    
    # Jitter circulaire pour MST aussi
    mst_jitter_x, mst_jitter_y = circular_jitter(
        n_cells, clustering, node_sizes,
        max_radius=mst_scale * 0.8,  # Un peu moins que Grid car MST est plus espacé
        min_radius=mst_scale * 0.2
    )
    
    # Appliquer le jitter
    xNodes_jittered = xNodes_base + mst_jitter_x
    yNodes_jittered = yNodes_base + mst_jitter_y
    
    # Décaler pour que les axes commencent à 1 (X ET Y)
    xNodes = xNodes_jittered - xNodes_jittered.min() + 1.0
    yNodes = yNodes_jittered - yNodes_jittered.min() + 1.0
    
    df_export['xNodes'] = xNodes.astype(np.float32)
    df_export['yNodes'] = yNodes.astype(np.float32)
    
    print(f"✅ xNodes: [{xNodes.min():.3f} - {xNodes.max():.3f}]")
    print(f"✅ yNodes: [{yNodes.min():.3f} - {yNodes.max():.3f}]")

# =====================================================================
# TAILLE DES NODES (pour chaque cellule)
# =====================================================================
size_col = np.array([node_sizes[int(c)] for c in clustering], dtype=np.float32)
df_export['size'] = size_col
print(f"✅ size: [{size_col.min():.0f} - {size_col.max():.0f}]")

# =====================================================================
# EXPORT CSV
# =====================================================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_path = os.path.join(OUTPUT_DIR, f"flowsom_results_{timestamp}.csv")
df_export.to_csv(csv_path, index=False)

print(f"\n✅ CSV exporté: {csv_path}")
print(f"   Shape: {df_export.shape}")

# =====================================================================
# EXPORT FCS COMPATIBLE KALUZA
# =====================================================================
print("\n" + "="*70)
print("📄 EXPORT FCS COMPATIBLE KALUZA")
print("="*70)

def export_to_fcs_kaluza(df, output_path):
    """Export FCS compatible Kaluza avec toutes les coordonnées positives."""
    try:
        import fcswrite
        
        numeric_df = df.select_dtypes(include=[np.number])
        data = numeric_df.values.astype(np.float32)
        channels = list(numeric_df.columns)
        
        # Nettoyer NaN/Inf
        data = np.nan_to_num(data, nan=0.0, posinf=1e6, neginf=0.0)
        
        print(f"   {data.shape[0]:,} events, {data.shape[1]} canaux")
        
        fcswrite.write_fcs(output_path, channels, data, compat_chn_names=True)
        return True
        
    except ImportError:
        print("   ⚠️ fcswrite non disponible (pip install fcswrite)")
        return False
    except Exception as e:
        print(f"   ⚠️ Erreur: {e}")
        return False

# Préparer le DataFrame FCS
df_fcs = df_export.select_dtypes(include=[np.number]).copy()

# Vérifier les ranges
print(f"\n📋 Colonnes exportées vers FCS:")
for col in ['FlowSOM_metacluster', 'FlowSOM_cluster', 'xGrid', 'yGrid', 'xNodes', 'yNodes', 'size', 'Condition_Num']:
    if col in df_fcs.columns:
        print(f"   ✅ {col:25s}: [{df_fcs[col].min():10.2f}, {df_fcs[col].max():10.2f}]")

# Export
fcs_path = os.path.join(OUTPUT_DIR, f"flowsom_results_{timestamp}.fcs")
if export_to_fcs_kaluza(df_fcs, fcs_path):
    print(f"\n✅ FCS exporté: {fcs_path}")
    print(f"\n💡 Dans Kaluza/FlowJo:")
    print(f"   - xGrid vs yGrid → visualisation grille SOM (cercles style FlowSOM R)")
    print(f"   - xNodes vs yNodes → visualisation arbre MST")
    print(f"   - La taille des cercles = proportionnelle au nombre de cellules")
    print(f"   - Colorez par FlowSOM_metacluster")

In [ ]:
# =============================================================================
# EXPORT DU RAPPORT DE STATISTIQUES
# =============================================================================

# Sauvegarder le rapport de statistiques
stats_path = os.path.join(OUTPUT_DIR, f"flowsom_statistics_{timestamp}.csv")
df_stats.to_csv(stats_path, index=False)
print(f"✅ Statistiques exportées: {stats_path}")

# Sauvegarder la matrice MFI
mfi_path = os.path.join(OUTPUT_DIR, f"flowsom_mfi_matrix_{timestamp}.csv")
df_mfi.to_csv(mfi_path)
print(f"✅ Matrice MFI exportée: {mfi_path}")

# Résumé final
print("\n" + "="*80)
print("📋 RÉSUMÉ DE L'ANALYSE FLOWSOM")
print("="*80)
print(f"   Fichiers analysés: {len(all_adatas)}")
print(f"   Cellules totales: {len(cell_data)}")
print(f"   Marqueurs utilisés: {len(used_markers)}")
print(f"   Nombre de métaclusters: {N_CLUSTERS}")
print(f"   Transformation: {TRANSFORM_TYPE}")
print(f"   Cofacteur: {COFACTOR}")
print("="*80)
print("✅ Analyse FlowSOM terminée avec succès!")